In [5]:
import pandas as pd
from geopy.distance import vincenty
import time

In [2]:
#订单流之间的关系
rawDataDir_flow2flow_info='/Users/leidelong/work/projects/knowledge_graph/2002445_flow2flow_info.txt'
#订单信息
rawDataDir_waybill_info='/Users/leidelong/work/projects/knowledge_graph/2002445_1month.txt'

df_flow2flow=pd.read_csv(rawDataDir_flow2flow_info,sep=',')
df_waybill=pd.read_csv(rawDataDir_waybill_info,sep='	')


df_waybill=df_waybill[ (df_waybill['poi_class_id'] >=0 ) & (  df_waybill['recipient_class_id'] >=0 )]
print df_waybill.shape
print df_waybill.columns

df_waybill=df_waybill[(df_waybill['ctime_hour']>=11) & (df_waybill['ctime_hour']<=13)]
df_waybill.sort_values(["ctime"],ascending=True)
print df_waybill.shape

(181411, 15)
Index([u'bm_waybill_id', u'bm_delivery_area_id', u'rider_id', u'recipient_lng',
       u'recipient_lat', u'platform_poi_id', u'ctime', u'finished_time',
       u'delivery_distance', u'delivery_distance_source', u'poi_class_id',
       u'recipient_class_id', u'distance', u'wb_dt', u'ctime_hour'],
      dtype='object')
(53479, 15)


In [3]:
df_waybill.head()

,bm_waybill_id,bm_delivery_area_id,rider_id,recipient_lng,recipient_lat,platform_poi_id,ctime,finished_time,delivery_distance,delivery_distance_source,poi_class_id,recipient_class_id,distance,wb_dt,ctime_hour
22,1506222125324548,2002445,2288514,121571650,31094740,2584046,1506222125,1506224506,2921.0,3.0,1.0,4,505.251489,20170924,11
23,1506222292020872,2002445,2885563,121591150,31074680,2665905,1506222292,1506226106,5017.0,3.0,1.0,2,567.300818,20170924,11
24,1506222524063088,2002445,3515368,121577172,31108126,2472327,1506222524,1506225811,2195.0,3.0,2.0,3,239.568700,20170924,11
25,1506222661175880,2002445,3558676,121587083,31068676,2179282,1506222661,1506225264,4183.0,3.0,2.0,1,372.503337,20170924,11
26,1506222677011056,2002445,706856,121604839,31137332,1329428,1506222677,1506225184,3827.0,3.0,1.0,34,737.235059,20170924,11


In [13]:
df_waybill['minutes']=df_waybill['ctime'].apply(lambda x :time.localtime(x).tm_hour*60 +  time.localtime(x).tm_min)

In [19]:
df_waybill.head()

,bm_waybill_id,bm_delivery_area_id,rider_id,recipient_lng,recipient_lat,platform_poi_id,ctime,finished_time,delivery_distance,delivery_distance_source,poi_class_id,recipient_class_id,distance,wb_dt,ctime_hour,minutes
22,1506222125324548,2002445,2288514,121571650,31094740,2584046,1506222125,1506224506,2921.0,3.0,1.0,4,505.251489,20170924,11,662
23,1506222292020872,2002445,2885563,121591150,31074680,2665905,1506222292,1506226106,5017.0,3.0,1.0,2,567.300818,20170924,11,664
24,1506222524063088,2002445,3515368,121577172,31108126,2472327,1506222524,1506225811,2195.0,3.0,2.0,3,239.568700,20170924,11,668
25,1506222661175880,2002445,3558676,121587083,31068676,2179282,1506222661,1506225264,4183.0,3.0,2.0,1,372.503337,20170924,11,671
26,1506222677011056,2002445,706856,121604839,31137332,1329428,1506222677,1506225184,3827.0,3.0,1.0,34,737.235059,20170924,11,671


In [25]:
df_waybill_baseinfo=df_waybill.loc[:,['bm_delivery_area_id','poi_class_id','recipient_class_id','minutes']]


In [47]:
df_waybillflow_waybillcount=df_waybill_baseinfo.groupby(['bm_delivery_area_id','poi_class_id','recipient_class_id','minutes']).size().reset_index(name='counts')

df_waybillflow_waybillcount.head()


,bm_delivery_area_id,poi_class_id,recipient_class_id,minutes,counts
0,2002445,0.0,1,666,1
1,2002445,0.0,1,674,2
2,2002445,0.0,1,683,1
3,2002445,0.0,1,689,1
4,2002445,0.0,1,693,1


In [32]:
print df_flow2flow.columns
print df_flow2flow.shape

Index([u'distance_mode1', u'distance_mode2', u'distance_mode3',
       u'end_flow_distance', u'end_flow_poi_class_center_lat',
       u'end_flow_poi_class_center_lng', u'end_flow_poi_class_id',
       u'end_flow_recipient_class_center_lat',
       u'end_flow_recipient_class_center_lng', u'end_flow_recipient_class_id',
       u'poiclass2poiclass_distance', u'start_flow_distance',
       u'start_flow_poi_class_center_lat', u'start_flow_poi_class_center_lng',
       u'start_flow_poi_class_id', u'start_flow_recipient_class_center_lat',
       u'start_flow_recipient_class_center_lng',
       u'start_flow_recipient_class_id', u'userclass2userclass_distance'],
      dtype='object')
(57600, 19)


In [34]:
df_flow2flow=df_flow2flow[ (df_flow2flow['userclass2userclass_distance'] <2000 ) & (df_flow2flow['start_flow_poi_class_id']==df_flow2flow['end_flow_poi_class_id']) ]

In [35]:
df_flow2flow.shape

(1200, 19)

In [37]:
df_flow2flow.head()

,distance_mode1,distance_mode2,distance_mode3,end_flow_distance,end_flow_poi_class_center_lat,end_flow_poi_class_center_lng,end_flow_poi_class_id,end_flow_recipient_class_center_lat,end_flow_recipient_class_center_lng,end_flow_recipient_class_id,poiclass2poiclass_distance,start_flow_distance,start_flow_poi_class_center_lat,start_flow_poi_class_center_lng,start_flow_poi_class_id,start_flow_recipient_class_center_lat,start_flow_recipient_class_center_lng,start_flow_recipient_class_id,userclass2userclass_distance
0,0.666667,2.000000,2.000000,6332.309519,31.102557,121.609862,0,31.101278,121.543504,0,0.0,6332.309519,31.102557,121.609862,0,31.101278,121.543504,0,0.000000
241,0.666667,2.000000,2.000000,4687.994910,31.102557,121.609862,0,31.066597,121.584017,1,0.0,4687.994910,31.102557,121.609862,0,31.066597,121.584017,1,0.000000
242,0.629746,1.276392,1.645886,3285.580290,31.102557,121.609862,0,31.075040,121.597080,2,0.0,4687.994910,31.102557,121.609862,0,31.066597,121.584017,1,1558.968333
481,0.708186,1.645886,1.276392,4687.994910,31.102557,121.609862,0,31.066597,121.584017,1,0.0,3285.580290,31.102557,121.609862,0,31.075040,121.597080,2,1558.968333
482,0.666667,2.000000,2.000000,3285.580290,31.102557,121.609862,0,31.075040,121.597080,2,0.0,3285.580290,31.102557,121.609862,0,31.075040,121.597080,2,0.000000


In [40]:
df_flow2flow_start=df_flow2flow.loc[:,['start_flow_poi_class_id','start_flow_recipient_class_id']].drop_duplicates()

In [43]:
df_flow2flow_start.shape

(240, 2)

In [ ]:
for i in df_flow2flow_start.index:
    start_flow_poi_class_id = df_flow2flow_start.at[i, "start_flow_poi_class_id"]
    start_flow_recipient_class_id = df_flow2flow_start.at[i, "start_flow_recipient_class_id"]
    
    #顺路订单
    follow_group=df_flow2flow[(df_flow2flow['start_flow_poi_class_id']==start_flow_poi_class_id)&
                              (df_flow2flow['start_flow_recipient_class_id']==start_flow_recipient_class_id)]
    
    
    for j in df_flow2flow.index:
        waybill_count = df_flow2flow.at[flow_index_next, "poi_class_center_lng"]
    